[Boruta Algorithm Explained](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a)

[Boruta implementation](https://towardsdatascience.com/feature-selection-with-boruta-in-python-676e3877e596)

[Feature Selection with Boruta package](https://www.datacamp.com/tutorial/feature-selection-R-boruta)

In [57]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '/Users/eliassovikgunnarsson/Master/windowing_and_cv')
from window_generator import WindowGenerator

In [58]:
df = pd.read_csv('features.csv', index_col=0)
df.head()

,VIX,US_EPU,DFF,DPRIME,DAAA,DBAA,T10Y3M,UNRATE,CPIAUCSL,PSAVERT,WTI,BRENT_EU,DEXJPUS,DUMMY
1990-01-02,17.24,169.23,8.54,10.5,8.87,9.85,0.11,5.4,127.5,8.0,22.88,21.20,146.25,0
1990-01-03,18.19,59.10,8.37,10.5,8.90,9.86,0.10,5.4,127.5,8.0,23.81,22.65,145.70,1
1990-01-04,19.22,76.38,8.29,10.5,8.89,9.84,0.14,5.4,127.5,8.0,23.41,22.50,143.37,2
1990-01-05,20.11,57.82,8.20,10.5,8.88,9.88,0.20,5.4,127.5,8.0,23.07,23.13,143.82,3
1990-01-06,20.11,218.48,8.20,10.5,8.88,9.88,0.20,5.4,127.5,8.0,23.07,23.13,143.82,4


In [59]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [60]:
w = WindowGenerator(input_width=3, label_width=1, shift=1, label_columns=['VIX'], train_df=train_df, val_df=val_df, test_df=test_df)
w

Total window size: 4
Input indices: [0 1 2]
Label indices: [3]
Label column name(s): ['VIX']

In [61]:
train = w.train

x_train = np.concatenate([x for x, y in train], axis = 0)
y_train = np.concatenate([y for x, y in train], axis = 0)


In [62]:
x_train.shape

(8508, 3, 14)

In [63]:
x_train = x_train.reshape((len(x_train), -1))
x_train[0]

array([1.7240e+01, 1.6923e+02, 8.5400e+00, 1.0500e+01, 8.8700e+00,
       9.8500e+00, 1.1000e-01, 5.4000e+00, 1.2750e+02, 8.0000e+00,
       2.2880e+01, 2.1200e+01, 1.4625e+02, 0.0000e+00, 1.8190e+01,
       5.9100e+01, 8.3700e+00, 1.0500e+01, 8.9000e+00, 9.8600e+00,
       1.0000e-01, 5.4000e+00, 1.2750e+02, 8.0000e+00, 2.3810e+01,
       2.2650e+01, 1.4570e+02, 1.0000e+00, 1.9220e+01, 7.6380e+01,
       8.2900e+00, 1.0500e+01, 8.8900e+00, 9.8400e+00, 1.4000e-01,
       5.4000e+00, 1.2750e+02, 8.0000e+00, 2.3410e+01, 2.2500e+01,
       1.4337e+02, 2.0000e+00], dtype=float32)

In [64]:
y_train = y_train.reshape((-1, ))
y_train.shape

(8508,)

Need to store label indices

In [65]:
cols = df.columns
flat_cols = []

for i in range(w.input_width, 0, -1):
    lag = f'_Lag_{i}'
    for col in cols:
        flat_cols.append(f'{col}{lag}')

flat_column_indices = {name: i for i, name in enumerate(flat_cols)}

In [66]:
flat_column_indices

{'VIX_Lag_3': 0,
 'US_EPU_Lag_3': 1,
 'DFF_Lag_3': 2,
 'DPRIME_Lag_3': 3,
 'DAAA_Lag_3': 4,
 'DBAA_Lag_3': 5,
 'T10Y3M_Lag_3': 6,
 'UNRATE_Lag_3': 7,
 'CPIAUCSL_Lag_3': 8,
 'PSAVERT_Lag_3': 9,
 'WTI_Lag_3': 10,
 'BRENT_EU_Lag_3': 11,
 'DEXJPUS_Lag_3': 12,
 'DUMMY_Lag_3': 13,
 'VIX_Lag_2': 14,
 'US_EPU_Lag_2': 15,
 'DFF_Lag_2': 16,
 'DPRIME_Lag_2': 17,
 'DAAA_Lag_2': 18,
 'DBAA_Lag_2': 19,
 'T10Y3M_Lag_2': 20,
 'UNRATE_Lag_2': 21,
 'CPIAUCSL_Lag_2': 22,
 'PSAVERT_Lag_2': 23,
 'WTI_Lag_2': 24,
 'BRENT_EU_Lag_2': 25,
 'DEXJPUS_Lag_2': 26,
 'DUMMY_Lag_2': 27,
 'VIX_Lag_1': 28,
 'US_EPU_Lag_1': 29,
 'DFF_Lag_1': 30,
 'DPRIME_Lag_1': 31,
 'DAAA_Lag_1': 32,
 'DBAA_Lag_1': 33,
 'T10Y3M_Lag_1': 34,
 'UNRATE_Lag_1': 35,
 'CPIAUCSL_Lag_1': 36,
 'PSAVERT_Lag_1': 37,
 'WTI_Lag_1': 38,
 'BRENT_EU_Lag_1': 39,
 'DEXJPUS_Lag_1': 40,
 'DUMMY_Lag_1': 41}

In [67]:
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

let's initialize a RF model 

In [68]:
model = RandomForestRegressor(n_estimators=250, max_depth=10, random_state=42)

let's initialize Boruta

In [69]:
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=20  # number of iterations to perform
)

In [70]:
feat_selector.fit(x_train, y_train)

Iteration: 	1 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	2 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	3 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	4 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	5 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	6 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	7 / 20
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	8 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	9 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	10 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	11 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	12 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	13 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	14 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	15 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	16 / 20
Confirmed: 	4
Tentative: 	1
Rejected: 	37
Iteration: 	17 / 

BorutaPy(estimator=RandomForestRegressor(max_depth=10, n_estimators=31,
                                         random_state=RandomState(MT19937) at 0x109E2C740),
         max_iter=20, n_estimators='auto',
         random_state=RandomState(MT19937) at 0x109E2C740, verbose=2)

In [71]:
# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", flat_cols[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              flat_cols[i], " - Ranking: ", feat_selector.ranking_[i])


------Support and Ranking for each feature------
Passes the test:  VIX_Lag_3  - Ranking:  1
Doesn't pass the test:  US_EPU_Lag_3  - Ranking:  14
Doesn't pass the test:  DFF_Lag_3  - Ranking:  12
Doesn't pass the test:  DPRIME_Lag_3  - Ranking:  37
Doesn't pass the test:  DAAA_Lag_3  - Ranking:  17
Doesn't pass the test:  DBAA_Lag_3  - Ranking:  21
Doesn't pass the test:  T10Y3M_Lag_3  - Ranking:  5
Doesn't pass the test:  UNRATE_Lag_3  - Ranking:  31
Doesn't pass the test:  CPIAUCSL_Lag_3  - Ranking:  35
Doesn't pass the test:  PSAVERT_Lag_3  - Ranking:  18
Doesn't pass the test:  WTI_Lag_3  - Ranking:  15
Doesn't pass the test:  BRENT_EU_Lag_3  - Ranking:  27
Doesn't pass the test:  DEXJPUS_Lag_3  - Ranking:  6
Doesn't pass the test:  DUMMY_Lag_3  - Ranking:  24
Passes the test:  VIX_Lag_2  - Ranking:  1
Doesn't pass the test:  US_EPU_Lag_2  - Ranking:  3
Doesn't pass the test:  DFF_Lag_2  - Ranking:  11
Doesn't pass the test:  DPRIME_Lag_2  - Ranking:  39
Doesn't pass the test:  DAA

In [72]:
feat_selector.ranking_

array([ 1, 14, 12, 37, 17, 21,  5, 31, 35, 18, 15, 27,  6, 24,  1,  3, 11,
       39, 24,  2, 12, 33, 36, 24, 29, 30, 15, 18,  1,  4,  7, 37,  8,  1,
       10, 31, 34, 26, 28, 20,  9, 22])